### Table of Contents

* [Step 1: Scope the Project and Gather Data](#step1)
    * [1.1 Determine Scope](#step1.1)
    * [1.2 Describe and Gather Data](#step1.2)
    
    
* [Step 2: Explore and Assess the Data](#step2)
    * [2.1 Explore the Data](#step2.1)
    * [2.2 Clean the Data](#step2.2)
    
    
* [Step 3: Define the Data Model](#step3)
    * [3.1 Design the Conceptual Data Model](#step3.1)    
    * [3.2 Map Out Data Pipelines](#step3.2)
    
    
* [Step 4: Run Pipelines to Model the Data](#step4)
    * [4.1 Create the Data Model](#step4.1)
    * [4.2 Run Data Quality Checks](#step4.2)
    * [4.3 Provide Data Dictionary](#step4.3)
    
    
* [Step 5: Complete Project Write-Up](#step5)

<img src="mariana_fruit_bat.jfif" alt="Juvenile Mariana Fruit Bat" width="300"/>

# Tracking the Vespertilionidae Family of Bats
### Data Engineering Capstone Project

#### Project Summary

I am interested in biology and zoology, so rather than work on the Udacity-provided data sets, I wanted to research data sets on my own to design a project that held some personal interest. I started by deciding that I wanted to learn more about how bats (specifically those in the Vespertilionidae family) are tracked for scientific studies and then researched data sets that would provide this information as well as more general data about these bats as a species.

In [ ]:
# Imports and installs

import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
import os
import hashlib
import csv
import json
import io
import boto3
from datetime import datetime, timedelta
import configparser
from configparser import ConfigParser

### Step 1: Scope the Project and Gather Data<a class="anchor" id="step1"></a>

#### 1.1 Determine Scope<a class="anchor" id="step1.1"></a>
I used three primary sets of data:

* GBIF occurrence data
* Movebank data
* List of vernacular (common) names


I imagined if I were a wildlife biologist or scientist, I might want to be able to quickly determine which species are being studied and tracked. I wanted to gather and combine the data sets I chose to show if any species that are listed in the GBIF occurrence data are included in any tracking studies. I used Amazon S3 to store the data and Amazon Redshift to design the database.




#### 1.2 Describe and Gather Data<a class="anchor" id="step1.2"></a>
I analyzed three primary sets of data:

* GBIF occurrence data
* Movebank data
* List of vernacular (common) names

The following text is copied from the GBIF website:


> GBIF—the Global Biodiversity Information Facility—is an international network and data infrastructure funded by the world's governments and aimed at providing anyone, anywhere, open access to data about all types of life on Earth.

> Coordinated through its Secretariat in Copenhagen, the GBIF network of participating countries and organizations, working through participant nodes, provides data-holding institutions around the world with common standards, best practices and open-source tools enabling them to share information about where and when species have been recorded. This knowledge derives from many sources, including everything from museum specimens collected in the 18th and 19th century to geotagged smartphone photos shared by amateur naturalists in recent days and weeks.

I used the GBIF database interface online to find occurrence data for the bats I was interested in. I drilled down through the scientific classifications to the Vespertilionidae family, which is a family of microbats. I then downloaded the resulting CSV file from the website to use in my analysis. This left me with 1,886,467 rows of data. The full list of fields was as follows:
* gbifID
* datasetKey
* occurrenceID
* kingdom
* phylum
* class
* order
* family
* genus
* species
* infraspecificEpithet
* taxonRank
* scientificName
* verbatimScientificName
* verbatimScientificNameAuthorship
* countryCode
* locality
* stateProvince
* occurrenceStatus
* individualCount
* publishingOrgKey
* decimalLatitude
* decimalLongitude
* coordinateUncertaintyInMeters
* coordinatePrecision
* elevation
* elevationAccuracy
* depth
* depthAccuracy
* eventDate
* day
* month
* year
* taxonKey
* speciesKey
* basisOfRecord
* institutionCode
* collectionCode
* catalogNumber
* recordNumber
* identifiedBy
* dateIdentified
* license
* rightsHolder
* recordedBy
* typeStatus
* establishmentMeans
* lastInterpreted
* mediaType
* issue

These fields had varying data types—some were objects, some were floats, some were integers. I knew I would not likely need all of the information, so I wanted to focus on the fields that seemed most relevant to my project.
I also used the animal tracking data provided by Movebank (https://www.movebank.org/cms/movebank-main), which is a free online database of animal tracking data hosted by the Max Planck Institute of Animal Behavior.

Movebank provides a public REST API for researchers. I followed the detailed instructions provided online, particularly the sample Python code, to gather the data for all studies included in the database. The raw data set contained the following fields:

* acknowledgements
* citation
* go_public_date
* grants_used
* has_quota
* i_am_owner
* id
* is_test
* license_terms
* license_type
* main_location_lat
* main_location_long
* name
* number_of_deployments
* number_of_individuals
* number_of_tags
* principal_investigator_address
* principal_investigator_email
* principal_investigator_name
* study_objective
* study_type
* suspend_license_terms
* i_can_see_data
* there_are_data_which_i_cannot_see
* i_have_download_access
* i_am_collaborator
* study_permission
* timestamp_first_deployed_location
* timestamp_last_deployed_location
* number_of_deployed_locations
* taxon_ids
* sensor_type_ids
* contact_person_name

I did change one line of code to be able to view all of the data: I set “true” for the “i_can_see_data” and “there_are_data_which_i_cannot_see” columns.


As with the GBIF data, these fields had varying data types—some were objects, some were floats, some were integers—and I reviewed the list of columns and a sample of the data to find the most relevant information.


To find the vernacular names of these bat species, I used the Integrated Taxonomic Information System (ITIS) online.


The Integrated Taxonomic Information System contains authoritative taxonomic information on plants, animals, fungi, and microbes of North America and the world. It is a partnership of U.S., Canadian, and Mexican agencies (ITIS-North America); other organizations; and taxonomic specialists. ITIS is also a partner of Species 2000 and the Global Biodiversity Information Facility (GBIF). The ITIS and Species 2000 Catalogue of Life (CoL) partnership provides the taxonomic backbone to the Encyclopedia of Life (EOL).



I searched for common names containing “bat” in every kingdom. I then copied and pasted the list into an Excel file. I had to do quite a bit of pre-processing for the data in Excel before doing any type of analysis. I formatted the columns, deleted any extraneous information (such as entries for “wombat” or other irrelevant names as well as the researcher name and year), concatenated text as appropriate, formatted the text as proper case, removed duplicates, and cleaned up extra spacing. I also sorted the scientific names chronologically and concatenated any common names if the species had more than one common name listed for that species. I then spell-checked the list, cleaned up punctuation, and fixed typos (one entry had “trumped-eared bat” instead of “trumpet-eared bat”). I saved the Excel file as a CSV file to use in the Jupyter notebook.

After analyzing these data sets to get a better idea of which columns I might need and what format they were in, I started developing a preliminary data model using a star schema. I decided to use Amazon S3 and Redshift to store and analyze the data.

In [ ]:
# Read in GBIF file

df_gbif = pd.read_csv('GBIF_Bat_Occurrences.csv', sep='\t', error_bad_lines=False)

df_gbif.head()

In [ ]:
df_gbif.columns

In [ ]:
df_gbif.shape

In [ ]:
# Read in Vernacular Names file

df_vern_name = pd.read_csv('Bats_Vernacular_Names.csv', encoding = 'unicode_escape')

df_vern_name.head()

In [ ]:
df_vern_name.describe()

In [ ]:
df_vern_name.shape

In [1]:
# Import Movebank data

def callMovebankAPI(params):
    # Requests Movebank API with ((param1, value1), (param2, value2),).
    # Assumes the environment variables 'mbus' (Movebank user name) and 'mbpw' (Movebank password).
    # Returns the API response as plain text.

    parser = configparser.ConfigParser()
    parser.read('movebank_config.ini')
    movebank_usrnm = parser.get('movebank_auth', 'movebank_username')
    movebank_pwd = parser.get('movebank_auth', 'movebank_password')

    response = requests.get('https://www.movebank.org/movebank/service/direct-read', params=params,
                            auth=HTTPBasicAuth(movebank_usrnm, movebank_pwd))
    print("Request " + response.url)
    if response.status_code == 200:  # successful request
        if 'License Terms:' in str(response.content):
            # only the license terms are returned, hash and append them in a subsequent request.
            # See also
            # https://github.com/movebank/movebank-api-doc/blob/master/movebank-api.md#read-and-accept-license-terms-using-curl
            print("Has license terms")
            hash = hashlib.md5(response.content).hexdigest()
            params = params + (('license-md5', hash),)
            # also attach previous cookie:
            response = requests.get('https://www.movebank.org/movebank/service/direct-read', params=params,
                                    cookies=response.cookies, auth=HTTPBasicAuth(movebank_usrnm, movebank_pwd))
            if response.status_code == 403:  # incorrect hash
                print("Incorrect hash")
                return ''
        return response.content.decode('utf-8')
    print(str(response.content))
    return ''

In [ ]:
def getStudies():
    studies = callMovebankAPI((('entity_type', 'study'), ('i_can_see_data', 'true'), ('there_are_data_which_i_cannot_see', 'true')))
    if len(studies) > 0:
        # parse raw text to dicts
        studies = csv.DictReader(io.StringIO(studies), delimiter=',')
        return [s for s in studies if s['i_can_see_data'] == 'true' or s['there_are_data_which_i_cannot_see'] == 'true']
    return []


if __name__ == "__main__":
    allstudies = getStudies()

In [ ]:
df_movebank_all = pd.DataFrame.from_records(allstudies)

df_movebank_all

In [ ]:
df_movebank_all.head()

In [ ]:
df_movebank_all.columns

In [ ]:
df_movebank_all.describe()

In [ ]:
df_movebank_all.shape

### Step 2: Explore and Assess the Data<a class="anchor" id="step2"></a>
#### 2.1 Explore the Data<a class="anchor" id="step2.1"></a>
To clean the data, I identified how the pandas dataframes would need to be combined or split to create the final staging table, identified any missing values, identified any duplicates, and identified and fixed any data types that were not compatible with the proposed data model.

#### 2.2 Clean the Data<a class="anchor" id="step2.2"></a>
The following list briefly outlines the steps I followed to clean the data:

* Split Movebank taxon ids into multiple rows (these were originally in one row separated by commas)
* Created a new pandas dataframe with only relevant columns for the Movebank data
* Merged the two dataframes
* Created a new dataframe with only relevant columns for the GBIF data
* Concatenated the Movebank and GBIF dataframes to create the main staging table
* Merged main staging dataframe and vernacular names dataframe to add a column for the vernacular names
* Checked for missing values (any missing values were converted to 0 [fill N/A with 0])
* Checked for duplicates (none)
* Fixed incorrect data types
* Changed gbifID to int 
* Changed taxonKey to int
* Changed id to int
* Changed day, month, and year to int
* Changed eventDate to date
* Reviewed list of columns and head for final staging table dataframe

In [ ]:
def process_clean_up(df1, df2, df3):
    
    # Explode/split taxon IDs into multiple rows
    df_movebank_split = pd.DataFrame(df1['taxon_ids'].str.split(',').tolist(), index=df1['id']).stack()
    df_movebank_split = df_movebank_split.reset_index([0, 'id'])
    df_movebank_split.columns = ['id', 'taxon_ids']
    
    # Create new dataframe with only needed columns
    df_movebank_subset = df1[['id', 'name', 'study_objective', 'study_type', 'citation', 'taxon_ids']]
    
    # Create new dataframe to merge two dataframes so taxon IDs are split
    df1 = pd.merge(df_movebank_split, df_movebank_subset, how='left', on=['id', 'taxon_ids'])
    
    # Create new dataframe with only needed columns for GBIF dataframe
    df_gbif_subset = df2[['taxonKey', 'species', 'gbifID', 'stateProvince', 'countryCode', 'decimalLatitude',
                          'decimalLongitude', 'eventDate', 'day', 'month', 'year']]
    
    # Concatenate Movebank and GBIF dataframes to create main staging table

    # Rename column in Movebank dataframe
    df1 = df1.rename(columns={'taxon_ids': 'species'})
    
    # Concatenate Movebank and GBIF dataframes
    df_main_staging = pd.concat(
                      [df_gbif_subset, df1],
                      join='outer',
                      ignore_index=True,
                      verify_integrity=True)
    
    # Add column for vernacular names
    
    # Rename columns
    df3 = df3.rename(columns={'speciesName': 'species'})
    
    # Merge dataframes
    df_main_staging_complete = pd.merge(df_main_staging, df3,
                                        how='outer',
                                        on='species')
    
    # Check for missing values
    df_main_staging_complete.isnull().sum
    
    # Check for duplicates
    df_main_staging_complete.duplicated()
    df_main_staging_complete.loc[df_main_staging_complete.duplicated(), :]
    
    # Fill missing values with 0 and change data type to int
    df_main_staging_complete['gbifID'] = df_main_staging_complete['gbifID'].fillna(0).astype(int)
    df_main_staging_complete['taxonKey'] = df_main_staging_complete['taxonKey'].fillna(0).astype(int)
    df_main_staging_complete['id'] = df_main_staging_complete['id'].fillna(0).astype(int)
    df_main_staging_complete['day'] = df_main_staging_complete['day'].fillna(0).astype(int)
    df_main_staging_complete['month'] = df_main_staging_complete['month'].fillna(0).astype(int)
    df_main_staging_complete['year'] = df_main_staging_complete['year'].fillna(0).astype(int)
    
    # Change eventDate to date
    df_main_staging_complete['eventDate'] = pd.to_datetime(df_main_staging_complete['eventDate']).dt.date
    
    return df_main_staging_complete

In [ ]:
df_main_staging_complete = process_clean_up(df_movebank_all, df_gbif, df_vern_name)

In [ ]:
df_main_staging_complete.head()

### Step 3: Define the Data Model<a class="anchor" id="step3"></a>
#### 3.1 Design the Conceptual Data Model<a class="anchor" id="step3.1"></a>
The image below (created in Lucidchart) shows the conceptual data model. I chose to use a star schema because it is simple (no need for complex joins), it is optimized for reading data more quickly, and it uses simpler business logic to communicate to stakeholders and other database users.

#### 3.2 Map Out Data Pipelines<a class="anchor" id="step3.2"></a>
The following steps were used to pipeline the data into the chosen data model:
* Convert pandas dataframe to .csv file
* Save .csv file in S3 bucket
* Create new tables in Redshift database
* Create staging table
* Load data into final fact and dimension tables in Redshift

<img src="UdacityFinalProject_DataModel.png" alt="Data Model" width="1200"/>

In [ ]:
# Save new dataframe as .csv file

df_main_staging_complete.to_csv('staging_complete.csv', index=False)

In [ ]:
# Upload .csv file to S3 bucket

config = configparser.ConfigParser()
config.read('dwh.cfg')

s3 = boto3.client('s3', aws_access_key_id = config.get('S3', 'ACCESS_KEY'),
                  aws_secret_access_key = config.get('S3', 'SECRET_KEY'))

s3.upload_file('staging_complete.csv', 'udacityfinalprojectanimaltracking', 'staging_complete.csv')

print('Upload successful')

### Step 4: Run Pipelines to Model the Data<a class="anchor" id="step4"></a>
#### 4.1 Create the Data Model<a class="anchor" id="step4.1"></a>
Build the data pipelines to create the data model.

In [ ]:
# Create tables — separate file

%run create_tables.py

In [ ]:
# Run ETL data pipeline — separate file

%run etl.py

#### 4.2 Run Data Quality Checks<a class="anchor" id="step4.2"></a>
I designated distinct data types for all of the columns used in the database tables. I also frequently checked the number of rows for the resultant data using the pandas .shape property and also previewed the data using the pandas .head()  function. Once the data was loaded in Redshift, I previewed the schema for each table and ran SQL queries to perform count checks.

#### 4.3 Provide Data Dictionary<a class="anchor" id="step4.3"></a>
Please see the separate file for the data dictionary: UdacityFinalProject_DataDictionary.pdf

### Step 5: Complete Project Write-Up<a class="anchor" id="step5"></a>

The ETL successfully processed the result in the final data model as shown in the following screenshots.

The final fact table (factBatTracking) has 1,893,739 rows.

<img src="fact_total_count_query.png" width="300"/>
<img src="fact_total_count_result.png" width="300"/>

The following screenshots show a sample query to find the number of occurrences for each year after the year 2000.

<img src="year_query.png" width="500"/>
<img src="year_result.png" width="500"/>


The sample query below shows results for any studies associated with fruit bats listed in the fact table.


<img src="fruit_bat_query.png" width="500"/>
<img src="fruit_bat_result.png" width="500"/>


For this project, I chose to use an Amazon S3 bucket to store data files and Amazon Redshift as a database platform because I felt relatively comfortable using these tools and these seemed to fit my goals for this project (creating a cloud data warehouse using a star schema for analytics).
    
For this type of project, the data should likely be updated every week. This would provide users with enough up-to-date information for analysis. Every day would be overkill; every month would not be often enough.
 
*If the data were increased by 100×*, I would likely use a different platform such as Spark to work with such a huge amount of data, possibly using a combination of Spark and Apache Livy in an EMR cluster.

*If the data populated a dashboard that had to be updated on a daily basis by 7 a.m. every day*, I would probably use a workflow management platform such as Airflow to schedule and monitor data workflows. I would also need to create a script to update the GBIF CSV file or connect to the GBIF API.

*If the database needed to be accessed by 100+ people*, I would consider using an enterprise data architecture, such as MySQL or Oracle, but Redshift would likely still be a good choice.